In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
print(os.getcwd()) 


C:\Users\Karthikeyan


In [3]:
import pandas as pd

df_customers = pd.read_csv('Customers.csv', encoding='ISO-8859-1')
df_sales = pd.read_csv('Sales.csv')
df_products = pd.read_csv('Products.csv')
df_stores = pd.read_csv('Stores.csv')
df_exchange_rates = pd.read_csv('Exchange_Rates.csv')


merged_df = pd.merge(df_sales, df_customers, on='CustomerKey', how='left')
merged_df = pd.merge(merged_df, df_products, on='ProductKey', how='left')
merged_df = pd.merge(merged_df, df_stores, on='StoreKey', how='left')
merged_df = pd.merge(merged_df, df_exchange_rates, left_on=['Order Date', 'Currency Code'], right_on=['Date', 'Currency'], how='left')
merged_df.drop(columns=['Date', 'Currency'], inplace=True)

In [9]:
merged_df.to_csv('merged_data.csv', index=False)

In [5]:
#finding missing values
missing_values = merged_df.isnull().sum()
print(missing_values[missing_values > 0])

Delivery Date    49719
State Code          30
Square Meters    13165
dtype: int64


In [15]:
missing_state = (merged_df['State_x'] == 'Napoli') & (merged_df['State Code'].isnull())
merged_df.loc[missing_state, 'State Code'] = 'NA'

In [17]:
duplicates = merged_df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

Number of duplicate rows: 0


In [19]:
merged_df.rename(columns={'State_x': 'Customer_state', 'State_y': 'Store_state'}, inplace=True)
print(merged_df.columns)

Index(['Order Number', 'Line Item', 'Order Date', 'Delivery Date',
       'CustomerKey', 'StoreKey', 'ProductKey', 'Quantity', 'Currency Code',
       'Gender', 'Name', 'City', 'State Code', 'Customer_state', 'Zip Code',
       'Country_x', 'Continent', 'Birthday', 'Product Name', 'Brand', 'Color',
       'Unit Cost USD', 'Unit Price USD', 'SubcategoryKey', 'Subcategory',
       'CategoryKey', 'Category', 'Country_y', 'Store_state', 'Square Meters',
       'Open Date', 'Exchange'],
      dtype='object')


In [21]:
merged_df['Unit Cost USD'] = merged_df['Unit Cost USD'].str.replace('[$,]', '', regex=True).astype(float)
merged_df['Unit Price USD'] = merged_df['Unit Price USD'].str.replace('[$,]', '', regex=True).astype(float)

merged_df['Gross_profit'] = merged_df['Unit Price USD'] - merged_df['Unit Cost USD']

merged_df['Gross_profit_margin (%)'] = (merged_df['Gross_profit'] / merged_df['Unit Price USD']) * 100

merged_df['Markup (%)'] = ((merged_df['Unit Price USD'] - merged_df['Unit Cost USD']) / merged_df['Unit Cost USD']) * 100

merged_df['Price_to_Cost_ratio'] = merged_df['Unit Price USD'] / merged_df['Unit Cost USD']

print(merged_df)

       Order Number  Line Item Order Date Delivery Date  CustomerKey  \
0            366000          1   1/1/2016           NaN       265598   
1            366001          1   1/1/2016     1/13/2016      1269051   
2            366001          2   1/1/2016     1/13/2016      1269051   
3            366002          1   1/1/2016     1/12/2016       266019   
4            366002          2   1/1/2016     1/12/2016       266019   
...             ...        ...        ...           ...          ...   
62879       2243030          1  2/20/2021           NaN      1216913   
62880       2243031          1  2/20/2021     2/24/2021       511229   
62881       2243032          1  2/20/2021     2/23/2021       331277   
62882       2243032          2  2/20/2021     2/23/2021       331277   
62883       2243032          3  2/20/2021     2/23/2021       331277   

       StoreKey  ProductKey  Quantity Currency Code  Gender  ...  \
0            10        1304         1           CAD    Male  ...   

In [23]:
merged_df.to_csv('merged_df.csv', index=False)

In [27]:
#z-score test
from scipy import stats
import numpy as np

numeric_cols = merged_df.select_dtypes(include=[np.number])
z_scores = np.abs(stats.zscore(numeric_cols))
outliers = (z_scores > 3)
print(outliers)

       Order Number  Line Item  CustomerKey  StoreKey  ProductKey  Quantity  \
0             False      False        False     False       False     False   
1             False      False        False     False       False     False   
2             False      False        False     False       False     False   
3             False      False        False     False       False     False   
4             False      False        False     False       False     False   
...             ...        ...          ...       ...         ...       ...   
62879         False      False        False     False       False     False   
62880         False      False        False     False       False     False   
62881         False      False        False     False       False     False   
62882         False      False        False     False       False     False   
62883         False      False        False     False       False     False   

       Unit Cost USD  Unit Price USD  SubcategoryKe

In [29]:
columns_to_remove = ['Exchange','Gross_profit_margin (%)','Markup (%)','Price_to_Cost_ratio','Continent','City','Name','Currency Code','Line Item'] 
df_filtered = merged_df.drop(columns=columns_to_remove)


In [31]:
import pandas as pd


df_filtered = df_filtered.rename(columns={
    'Country_x': 'Customer_country',
    'Country_y': 'Store_country',
   })

# Convert 'Order_Date' and 'Birthday' to datetime
df_filtered['Order Date'] = pd.to_datetime(df_filtered['Order Date'], errors='coerce')
df_filtered['Birthday'] = pd.to_datetime(df_filtered['Birthday'], errors='coerce')

# Calculate customer age in years
df_filtered['customer_age'] = (df_filtered['Order Date'] - df_filtered['Birthday']).dt.days // 365

# Check the updated DataFrame
print(df_filtered)

       Order Number Order Date Delivery Date  CustomerKey  StoreKey  \
0            366000 2016-01-01           NaN       265598        10   
1            366001 2016-01-01     1/13/2016      1269051         0   
2            366001 2016-01-01     1/13/2016      1269051         0   
3            366002 2016-01-01     1/12/2016       266019         0   
4            366002 2016-01-01     1/12/2016       266019         0   
...             ...        ...           ...          ...       ...   
62879       2243030 2021-02-20           NaN      1216913        43   
62880       2243031 2021-02-20     2/24/2021       511229         0   
62881       2243032 2021-02-20     2/23/2021       331277         0   
62882       2243032 2021-02-20     2/23/2021       331277         0   
62883       2243032 2021-02-20     2/23/2021       331277         0   

       ProductKey  Quantity  Gender State Code     Customer_state  ...  \
0            1304         1    Male         ON            Ontario  ...   

In [33]:
df_filtered.to_csv('Global_data.csv', index=False)

In [55]:
print(df_filtered)

       Order Number Order Date Delivery Date  CustomerKey  StoreKey  \
0            366000 2016-01-01          None       265598        10   
1            366001 2016-01-01     1/13/2016      1269051         0   
2            366001 2016-01-01     1/13/2016      1269051         0   
3            366002 2016-01-01     1/12/2016       266019         0   
4            366002 2016-01-01     1/12/2016       266019         0   
...             ...        ...           ...          ...       ...   
62879       2243030 2021-02-20          None      1216913        43   
62880       2243031 2021-02-20     2/24/2021       511229         0   
62881       2243032 2021-02-20     2/23/2021       331277         0   
62882       2243032 2021-02-20     2/23/2021       331277         0   
62883       2243032 2021-02-20     2/23/2021       331277         0   

       ProductKey  Quantity  Gender State Code     Customer_state  ...  \
0            1304         1    Male         ON            Ontario  ...   

In [57]:
import mysql.connector
import pandas as pd
df_filtered.columns = df_filtered.columns.str.replace(' ', '_')


print("Modified column names:")
print(df_filtered.columns.tolist())

Modified column names:
['Order_Number', 'Order_Date', 'Delivery_Date', 'CustomerKey', 'StoreKey', 'ProductKey', 'Quantity', 'Gender', 'State_Code', 'Customer_state', 'Zip_Code', 'Customer_country', 'Birthday', 'Product_Name', 'Brand', 'Color', 'Unit_Cost_USD', 'Unit_Price_USD', 'SubcategoryKey', 'Subcategory', 'CategoryKey', 'Category', 'Store_country', 'Store_state', 'Square_Meters', 'Open_Date', 'Gross_profit', 'customer_age']


In [66]:
import pandas as pd
import mysql.connector
# Fill missing values
df_filtered['Delivery_Date'] = df_filtered['Delivery_Date'].where(df_filtered['Delivery_Date'].notna(), None)  # Replace NaN with None (NULL)
df_filtered['Square_Meters'] = df_filtered['Square_Meters'].fillna(0)  # Replace NaN with 0


db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'project2'
}


try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()

    create_table_query = """
    CREATE TABLE IF NOT EXISTS your_table_name (
        Order_Number INT,
        Order_Date DATETIME,
        Delivery_Date DATETIME,
        CustomerKey INT,
        StoreKey INT,
        ProductKey INT,
        Quantity INT,
        Gender VARCHAR(10),
        State_Code VARCHAR(10),
        Customer_state VARCHAR(50),
        Zip_Code VARCHAR(10),
        Customer_country VARCHAR(50),
        Birthday DATE,
        Product_Name VARCHAR(100),
        Brand VARCHAR(50),
        Color VARCHAR(30),
        Unit_Cost_USD FLOAT,
        Unit_Price_USD FLOAT,
        SubcategoryKey INT,
        Subcategory VARCHAR(100),
        CategoryKey INT,
        Category VARCHAR(100),
        Store_country VARCHAR(50),
        Store_state VARCHAR(50),
        Square_Meters FLOAT,
        Open_Date DATE,
        Gross_profit FLOAT,
        customer_age INT
    )
    """
    cursor.execute(create_table_query)
    
    
    for index, row in df_filtered.iterrows():
        delivery_date = row['Delivery_Date'] if pd.notna(row['Delivery_Date']) else None  

        row_data = (
            row['Order_Number'], row['Order_Date'], delivery_date, row['CustomerKey'],
            row['StoreKey'], row['ProductKey'], row['Quantity'], row['Gender'],
            row['State_Code'], row['Customer_state'], row['Zip_Code'], 
            row['Customer_country'], row['Birthday'], row['Product_Name'], 
            row['Brand'], row['Color'], row['Unit_Cost_USD'], 
            row['Unit_Price_USD'], row['SubcategoryKey'], row['Subcategory'],
            row['CategoryKey'], row['Category'], row['Store_country'], 
            row['Store_state'], row['Square_Meters'], row['Open_Date'], 
            row['Gross_profit'], row['customer_age']
        )

        insert_query = """
        INSERT INTO your_table_name (
            Order_Number, Order_Date, Delivery_Date, CustomerKey, StoreKey,
            ProductKey, Quantity, Gender, State_Code, Customer_state,
            Zip_Code, Customer_country, Birthday, Product_Name, Brand,
            Color, Unit_Cost_USD, Unit_Price_USD, SubcategoryKey,
            Subcategory, CategoryKey, Category, Store_country,
            Store_state, Square_Meters, Open_Date, Gross_profit, customer_age
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        try:
            cursor.execute(insert_query, row_data)
        except mysql.connector.Error as e:
            print(f"Error inserting row: {row}\nError: {e}")

   
    connection.commit()
    
except mysql.connector.Error as e:
    print(f"Database connection error: {e}")
finally:
    
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [68]:
df_filtered.to_csv('df_filtered.csv', index=False)

In [51]:
import pandas as pd
import mysql.connector


db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'project2'
}


try:
    conn = mysql.connector.connect(**db_config)


    query = "SELECT * FROM your_table_name;"
    df = pd.read_sql(query, conn)

 
    csv_file_path = 'your_table.csv'
    df.to_csv(csv_file_path, index=False)
    print(f"Data has been exported to {csv_file_path}")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:

    if conn.is_connected():
        conn.close()

C:\Users\Karthikeyan\AppData\Local\Temp\ipykernel_16500\1310058846.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data has been exported to your_table.csv


In [89]:
#original
!pip install SQLAlchemy mysql-connector-python
from sqlalchemy import create_engine
import mysql.connector
from tabulate import tabulate


connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='project2'
)

cursor = connection.cursor()

from tabulate import tabulate

def execute_query(query, description, headers=None):
    cursor.execute(query)
    result = cursor.fetchall()
    
    print(f"\n{description}:")
    
    if headers:
        print(tabulate(result, headers=headers, tablefmt="grid"))
    else:
        print(tabulate(result, tablefmt="grid"))


import pandas as pd

query_1 = """
SELECT Customer_country, SUM(Gross_profit) AS Total_Gross_Profit
FROM your_table_name
GROUP BY Customer_country
ORDER BY Total_Gross_Profit DESC;
"""


cursor.execute(query_1)
result = cursor.fetchall()


df = pd.DataFrame(result, columns=["Country", "Total Gross Profit"])


df["Total Gross Profit"] = df["Total Gross Profit"].apply(lambda x: f'{x:,.2f}')


print("\nTotal Gross Profit by Country:")
print(tabulate(df.values, headers=df.columns, tablefmt="grid"))




query_2 = """
SELECT Customer_country, AVG(customer_age) AS Average_Age
FROM your_table_name
GROUP BY Customer_country
ORDER BY Average_Age DESC;
"""
execute_query(query_2, "Average Customer Age by Country", headers=["Country", "Average Age"])


query_3 = """
SELECT Subcategory, SUM(Quantity) AS Total_Quantity_Sold
FROM your_table_name
GROUP BY Subcategory
ORDER BY Total_Quantity_Sold DESC;
"""
execute_query(query_3, "Total Quantity Sold by Product_types", headers=["Product Type", "Total Quantity Sold"])


query_4 = """
SELECT Brand, SUM(Unit_Price_USD * Quantity) AS Total_Revenue
FROM your_table_name
GROUP BY Brand
ORDER BY Total_Revenue DESC;
"""


cursor.execute(query_4)
result = cursor.fetchall()


df = pd.DataFrame(result, columns=["Brand", "Total Revenue"])


df["Total Revenue"] = df["Total Revenue"].apply(lambda x: f'{x:,.2f}')


print("\nTotal Revenue by Brand:")
print(tabulate(df.values, headers=df.columns, tablefmt="grid"))


query_5 = """
SELECT Gender, Customer_country, COUNT(Order_Number) AS Number_of_Orders
FROM your_table_name
GROUP BY Gender, Customer_country
ORDER BY Number_of_Orders DESC;
"""
execute_query(query_5, "Orders by Gender and Country", headers=["Gender", "Country", "Number of Orders"])


query_6 = """
SELECT 
    CASE 
        WHEN StoreKey = 0 THEN 'Online Sales'
        ELSE 'Physical Stores'
    END AS Store_Type,
    SUM(Gross_profit) AS Total_Gross_Profit
FROM your_table_name
GROUP BY Store_Type;
"""


cursor.execute(query_6)
result = cursor.fetchall()


df = pd.DataFrame(result, columns=["Store Type", "Total Gross Profit"])


df["Total Gross Profit"] = df["Total Gross Profit"].apply(lambda x: f'{x:,.2f}')


print("\nTotal Gross Profit: Online vs. Physical Stores:")
print(tabulate(df.values, headers=df.columns, tablefmt="grid"))


query_7 = """
SELECT DATE_FORMAT(Order_Date, '%Y-%m') AS Month, SUM(Gross_profit) AS Total_Gross_Profit
FROM your_table_name
GROUP BY Month
ORDER BY Month;
"""
execute_query(query_7, "Monthly Sales Analysis", headers=["Month", "Total Gross Profit"])


query_8 = """
SELECT Product_Name, SUM(Quantity) AS Total_Units_Sold
FROM your_table_name
GROUP BY Product_Name
ORDER BY Total_Units_Sold DESC
LIMIT 5;
"""
execute_query(query_8, "Top 5 Best-Selling Products", headers=["Product Name", "Total Units Sold"])


query_top_product_age_group = """
WITH RankedProducts AS (
    SELECT 
        CASE 
            WHEN customer_age BETWEEN 0 AND 9 THEN '0-9'
            WHEN customer_age BETWEEN 10 AND 19 THEN '10-19'
            WHEN customer_age BETWEEN 20 AND 29 THEN '20-29'
            WHEN customer_age BETWEEN 30 AND 39 THEN '30-39'
            WHEN customer_age BETWEEN 40 AND 49 THEN '40-49'
            WHEN customer_age BETWEEN 50 AND 59 THEN '50-59'
            WHEN customer_age BETWEEN 60 AND 69 THEN '60-69'
            WHEN customer_age BETWEEN 70 AND 79 THEN '70-79'
            WHEN customer_age >= 80 THEN '80+'
            ELSE 'Unknown'
        END AS Age_Group,
        Product_Name,
        SUM(Quantity) AS Total_Units_Sold,
        ROW_NUMBER() OVER (PARTITION BY 
            CASE 
                WHEN customer_age BETWEEN 0 AND 9 THEN '0-9'
                WHEN customer_age BETWEEN 10 AND 19 THEN '10-19'
                WHEN customer_age BETWEEN 20 AND 29 THEN '20-29'
                WHEN customer_age BETWEEN 30 AND 39 THEN '30-39'
                WHEN customer_age BETWEEN 40 AND 49 THEN '40-49'
                WHEN customer_age BETWEEN 50 AND 59 THEN '50-59'
                WHEN customer_age BETWEEN 60 AND 69 THEN '60-69'
                WHEN customer_age BETWEEN 70 AND 79 THEN '70-79'
                WHEN customer_age >= 80 THEN '80+'
                ELSE 'Unknown'
            END 
            ORDER BY SUM(Quantity) DESC) AS Rank
    FROM your_table_name
    GROUP BY Age_Group, Product_Name
)
SELECT Age_Group, Product_Name, Total_Units_Sold
FROM RankedProducts
WHERE Rank = 1
ORDER BY Age_Group;
"""

cursor.execute(query_top_product_age_group)
result = cursor.fetchall()

df_top_product_age_group = pd.DataFrame(result, columns=["Age Group", "Product Name", "Total Units Sold"])


print("\nTop Product Preferred by Each Age Group:")
print(tabulate(df_top_product_age_group.values, headers=df_top_product_age_group.columns, tablefmt="grid"))

query = """
SELECT Store_state, SUM(Gross_profit) AS Total_Gross_Profit
FROM your_table_name
WHERE Store_state != 'Online'
GROUP BY Store_state
ORDER BY Total_Gross_Profit DESC
LIMIT 10;
"""

try:
    
    df = pd.read_sql(query, engine)

  
    print("Top 10 Stores based on Gross profit")
    print(df)

except Exception as err:
    print(f"Error: {err}")


cursor.close()
connection.close()



Total Gross Profit by Country:
+----------------+----------------------+
| Country        | Total Gross Profit   |
+================+======================+
| United States  | 5,862,024.74         |
+----------------+----------------------+
| United Kingdom | 1,431,849.73         |
+----------------+----------------------+
| Germany        | 1,038,006.76         |
+----------------+----------------------+
| Canada         | 946,569.60           |
+----------------+----------------------+
| Australia      | 538,863.68           |
+----------------+----------------------+
| Italy          | 517,512.85           |
+----------------+----------------------+
| Netherlands    | 381,486.25           |
+----------------+----------------------+
| France         | 307,932.24           |
+----------------+----------------------+

Average Customer Age by Country:
+----------------+---------------+
| Country        |   Average Age |
+================+===============+
| United States  |       49.879